# Lemke Howson: An algorithm to find Nash Equilibrium

This notebook introduces Lemke Howson algorithm for finding Nash equilibrium of a two-player strategic form game. 

In [1]:
import numpy as np

## Two-Player Games in Strategic Form

We are going to find Nash qquilibria (pure or mixed action) of a Two-Player Games $g = (I, (A_i)_{i \in I}, (u_i)_{i \in I})$, where 

* $I = \{0, 1\}$ is the set of players,
* $M = \{1, ..., m\}, N = \{m+1, ..., m+n\}$ are the action spaces,
* $\Delta^L = \{x \in \mathbb{R}^L_+ \mid \sum_{l \in L} x_l = 1\}, L=M, N$ are the mixed action spaces.
* $A \in \mathbb{R}^{M \times N}, B \in \mathbb{R}^{M \times N}$ are the payoff matrixs for player 0 and 1, respectively, and similarly the expected payoffs are $x^{\prime}Ay$ and $x^{\prime}By$.

### Nash Equilibrium

For $x \in \Delta^M$ and $y \in \Delta^N$, we define

$$
\bar{x} = \underset{j \in N}{\arg \max}(B^{\prime}x)_j, \quad
x^\circ = \{i \in M \mid x_i = 0\} \\
\bar{y} = \underset{i \in M}{\arg \max}(A^{\prime}y)_i, \quad
y^\circ = \{j \in N \mid y_j = 0\} \\
\text{supp}(x) = \{i \mid x_i > 0\}, \quad 
\text{supp}(y) = \{j \mid y_j > 0\}
$$

According to von Stengel, B. (2007), we can judge whether $(x, y) \in \Delta^M \times \Delta^N$ is a Nash equilibrium by checking:

* $\text{supp}(x) \subset \bar{y}, \text{supp}(y) \subset \bar{x}$, or
* $\bar{y} \cup x^\circ = M, \bar{x} \cup y^\circ = N$, equivalently $(\bar{x} \cup x^\circ) \cup (\bar{y} \cup y^\circ) = M \cup N$.

These two conditions are equivalent, so checking one is enough to find a Nash equilibrium.

Depending on these two conditions, we may use some algorithms to compute Nash equilibrium of any well-defined two-player strategic form game. However, we have to deal with some difficulties when facing degenerate games, as we will see later.

We first consider the case with nondegenerate game, which is simpler.

# Nondegenerate game

** Definition: ** A two-player game is nondegenerate if for any $x \in \Delta^M$ and any $y \in \Delta^N$, we have

$$
\left| \bar{x} \right| \leq \left| \text(supp)(x) \right|, \quad
\left| \bar{y} \right| \leq \left| \text(supp)(y) \right|,
$$

or equivalently,

$$
\left| x^\circ \right| + \left| \bar{x} \right| \leq m, \quad
\left| y^\circ \right| + \left| \bar{y} \right| \leq n.
$$

Combined with the condition a Nash equilibrium should satisfy described above, we know that if $(x, y)$ is a Nash equilibrium of a nondegenerate game, then

$$
\left| \text{supp}(x) \right| = \left| \text{supp}(y) \right|
$$

This reduces the number of possible support pairs for a Nash equilibrium a lot.

## Support Enumeration

Before we introduce Lemke Howson algorithm, we introduce a method to find all Nash equilibria by iterating all mixed actions with equal-sized support pairs, and checking whether they can be a Nash equilibrium. This is called Support Enumeration.

For each $k=1, ..., min\{m, n\}$ and each pair $(I, J)$, $I \subset M$ and $J \subset N$, such that $\left| I \right| = \left| J \right| = k$, the mixed action $(x, y)$ is a Nash equilibrium if it solves the systems of linear equations

$$
\sum_{j \in J} a_{ij} y_j = u \text{ for } i \in I, \quad
\sum_{j \in J} y_j = 1, \\
\sum_{i \in I} b_{ij} x_i = v \text{ for } j \in J, \quad
\sum_{i \in I} x_i = 1
$$

And also satisfies that
* $x_i > 0$ for all $i \in I$ and $y_j >0$ for all $j \in J$,
* $u \geq \sum_{j \in J} a_{ij} y_j$ for all $i \not \in I$ and $v \geq \sum_{i \in I} b_{ij} x_i$ for all $j \not \in J$.

The systems of equations can be written in matrix form as 

$$
C \begin{pmatrix} y_J \\ u \end{pmatrix} = e
$$
and
$$
D \begin{pmatrix} x_I \\ v \end{pmatrix} = e
$$
with
$$
C =
 \begin{pmatrix}
 A_{IJ} & -\mathbf{1} \\
 \mathbf{1}' & 0
 \end{pmatrix}, \quad
D =
 \begin{pmatrix}
 B'_{JI} & -\mathbf{1} \\
 \mathbf{1}' & 0
 \end{pmatrix}, \quad
e = \begin{pmatrix}\mathbf{0} \\ 1\end{pmatrix},
$$
where
$A_{IJ}$ is the submatrix of $A$ given by rows $I$ and columns $J$,
$B'_{JI}$ is the submatrix of $B'$ given by rows $J$ and columns $I$, and
$\mathbf{0}$ and $\mathbf{1}$ are the $k$-dimensional vectors of zeros and ones, respectively.

Using the algorithm described above, given a well-defined nondegenerate two-player game and a support pair $I, J$, we know whether there is a Nash equilirium corresponding to this support pair, and can calculate the NE $(x, y)$ if there is one.

To make it convenient, we denote the actions by indices, which starts with 0 in Python.

### Code

We first write a function which solves the system of linear equations for one player, which is the half of the whole system to find a Nash equilibiurm. In other word, it checks whether there exists a mixed action of opponent with support given, against which the actions in the player's own support are all best responses.

* The arguments are
  * numpy array `payoff_matrix`;
  * list (or numpy array) `own_supp` for the support of the player in consideration;
  * list (or numpy array) `opp_supp` for the support of the opponent player;
  * numpy array `out` that stores the candidate mixed action.
* If there is a mixed action of the opponent with support `opp_supp`
  against which the actions in `own_supp` are best responses,
  then return `True`; otherwise return `False`.
  In the former case, the mixed action is stored in `out`.
* Array `out` must be of length equal to the number of the opponent's actions.

In [4]:
def indiff_mixed_action(payoff_matrix, own_supp, opp_supp, out):
    # (number of own actions, number of opponent's actions)
    nums_actions = payoff_matrix.shape
    
    # Support size
    k = len(own_supp)
    
    # Matrix in the left hand side of the linear equation
    a = np.empty((k+1, k+1))
    a[:-1, :-1] = payoff_matrix[own_supp, :][:, opp_supp]
    a[-1, :-1] = 1
    a[:-1, -1] = -1
    a[-1, -1] = 0
    
    # Vector in the right hand side of the linear equation
    b = np.zeros(k+1)
    b[-1] = 1
    
    try:
        sol = np.linalg.solve(a, b)
    except np.linalg.LinAlgError:
        return False
    
    # Return False immediately if any of the "probabilities" is not positive
    if (sol[:-1] <= 0).any():
        return False
    
    own_supp_c = np.setdiff1d(np.arange(nums_actions[0]), own_supp)
    # Return False immediately if the solution mixed action is not optimal
    if (sol[-1] < payoff_matrix[own_supp_c, :][:, opp_supp] @ sol[:-1]).any():
        return False
    
    out.fill(0)
    out[opp_supp] = sol[:-1]
    return True

Given support $I$ and $J$, if for each player, the `indiff_mixed_action` returns `True`, then there is a Nash equilirium with support $I$ and $J$, and the mixed actions are just as calculated.

To find all Nash equilibria of a strategic form game, we just iterate all possible combinations of $I$ and $J$, and then apply `indiff_mixed_action` to each players with each support pair. If `indiff_mixed_action` returns `True` for both players, we store mixed actions in list `NEs`.

In [8]:
import itertools

def support_enumeration(A, B_T):
    """
    Given a nondegenerate m x n bimatrix game (A, B_T), return a list of
    all Nash equilibria computed by the support enumeration algorithm.
    
    Parameters
    ----------
    A : ndarray(float, ndim=2)
        Payoff matrix for player 1, of shape (m, n).
    
    B_T : ndarray(float, ndim=2)
        Payoff matrix for player 2, of shape (n, m).
        
    Returns
    -------
    NEs : list(tuple(ndarray(float, ndim=1)))
        List containing tuples of Nash equilibrium mixed actions.
    
    """
    m, n = A.shape
    NEs = []
    for k in range(1, min(m, n)+1):
        for I in itertools.combinations(range(m), k):
            for J in itertools.combinations(range(n), k):
                y = np.empty(n)
                if indiff_mixed_action(A, list(I), list(J), y):
                    x = np.empty(m)
                    if indiff_mixed_action(B_T, list(J), list(I), x):
                        NEs.append((x, y))
    return NEs

### Example

Consider the example from von Stengel (2007):
$$
A =
\begin{bmatrix}
3 & 3 \\
2 & 5 \\
0 & 6
\end{bmatrix},
\quad
B =
\begin{bmatrix}
3 & 2 \\
2 & 6 \\
3 & 1
\end{bmatrix}.
$$

The action spaces of players 1 and 2 are
$$
M = \{0, 1, 2\}, \quad
N = \{0, 1\}.
$$

Use `indiff_mixed_action` to find a Nash equilibrium, when $I = \{0, 1\}$ and $J = \{0, 1\}$.

In [13]:
# Define a two-player strategic form game
A = np.array([[3, 3],
              [2, 5],
              [0 ,6]])
B = np.array([[3, 2],
              [2, 6],
              [3, 1]])
m, n = A.shape  # Numbers of actions of players 1 and 2, respectively
M = np.arange(m)
N = np.arange(n)

# Set the equal-sized support pari I, J
I = [0, 1]
J = [0, 1]

out = np.empty(n)
indiff_mixed_action(A, I, J, out)

True

In [14]:
out

array([ 0.66666667,  0.33333333])

Use `support_enumeration`, we can find all Nash equilibria.

In [15]:
support_enumeration(A, B.T)

[(array([ 1.,  0.,  0.]), array([ 1.,  0.])),
 (array([ 0.8,  0.2,  0. ]), array([ 0.66666667,  0.33333333])),
 (array([ 0.        ,  0.33333333,  0.66666667]),
  array([ 0.33333333,  0.66666667]))]

In this game, there are 3 NEs.

Support Enumeration has one drawback, that the number of equal-sized support pairs increases quickly as the game gets larger:

If $m = n$, the number of equal-sized support pairs is 

$$
\sum^n_{k=1} \binom{n} {k} ^2 = \binom {2n} {n} - 1 \approx \frac{4^n}{\sqrt{\pi n}}
$$

We can see that the number of equal-sized support pairs increase exponentially. That's why Lemke-Howson algorithm is more desirable, which is less computational intensive.